## YΣ19 Artificial Intelligence II
# Homework 4

### Iglezou Myrto - 111520170038


<img src="https://venturebeat.com/wp-content/uploads/2020/03/CORD-19.png?w=1200&strip=all" alt="Cord-19" width="600"/>

# Project Description



The objective of this project is about developing a document retrieval system to return titles of scientific papers containing the answer to a given user question. The dataset used in this exercise is from [COVID-19 Open Research Dataset (CORD-19)](https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/historical_releases.html), the first version. We are gonna implement 2 different sentence embedding approaches, in order for the model to retrieve the titles of the papers related to a given question.


# **Question 1** 

## Step 1 - Preprocess the provided dataset

### Read all json files from folder and keep for dataset the title and the body. Then save the dataframe to a csv file for faster reading of the dataframe.

In [28]:
import io
import os
from google.colab import drive
import pandas as pd 
import json

drive.mount('/content/drive',force_remount=True)
path = r"/content/drive/My Drive/cord-19_2020-03-13/cord-19_2020-03-13/2020-03-13/comm_use_subset"

dataset_df = pd.DataFrame(columns=['id', 'title', 'body'])

for filename in os.listdir(path):
   with open(os.path.join(path, filename), 'r') as f:  
      json_text = json.load(f)

      id = json_text['paper_id']
      # print(id)
      title = json_text['metadata']['title']
      # print(title)
      body = json_text['body_text']
      # print(body)

      dataset_df.loc[len(dataset_df)] = [id,title,body]
   

Mounted at /content/drive


In [29]:
dataset_df.to_csv('dataset.csv',index=False)
!cp dataset.csv "drive/My Drive/"

### Read the dataset from the csv file and save the information in a dataframe

In [78]:
import io
from google.colab import drive
import pandas as pd 
import sys 

drive.mount('/content/drive',force_remount=True)
filePath = r"/content/drive/My Drive/dataset.csv"
dataset_df = pd.read_csv(filePath)
dataset_df.title = dataset_df.title.astype(str)  # make everything str, for lower() function

Mounted at /content/drive


**Dataframe before the preprocess**

In [79]:
dataset_df.head(5)

,id,title,body
0,236bd666a76213bc131969e1d5b66e410fc1cd45,MINI REVIEW Acute Phase Proteins in Marine Mam...,[{'text': 'The mammalian immune system include...
1,14374db205f6934d9cba148624462000bc6ec7be,Antibody Treatment against Angiopoietin-Like 4...,[{'text': 'IMPORTANCE Despite extensive global...
2,af678e8cd31d74cdb2d690addc19d59dca331f2b,Quantifying the seasonal drivers of transmissi...,"[{'text': ""Growing human population, urbanizat..."
3,42b049c2b5b32c094dc8b10f967e43ac2169b890,Evaluation of the influenza-like illness surve...,[{'text': 'the first evaluation of the Tunisia...
4,1664a9df618ca74e099245a2bd65f3172aeac284,nan,"[{'text': 'Worldwide, lung cancer remains the ..."


Remove some of the special characters, such as [, { , ' , : and some words form json like 'text'.

In [80]:
def removeCharacters(x):
  x = x.str.replace(r'\'text\'', '')
  x = x.str.replace(r'\'start\'', '')
  x = x.str.replace(r'\'end\'', '')
  x = x.str.replace(r'[{}]', '')
  x = x.str.replace(r'[\[\]]', '')
  x = x.str.replace(r'["]', '')
  x = x.str.replace(r'[\']', '')
  x = x.str.replace(r'[:]', '')
  return x

In [81]:
dataset_df['body'] = removeCharacters(dataset_df['body'])

Remove all the uppercase letters from title and body

In [82]:
dataset_df['body'] = dataset_df['body'].apply(lambda x: x.lower())
dataset_df['title'] = dataset_df['title'].apply(lambda x: x.lower())

**Dataframe after the preprocess**

In [83]:
dataset_df.head(5)

,id,title,body
0,236bd666a76213bc131969e1d5b66e410fc1cd45,mini review acute phase proteins in marine mam...,the mammalian immune system includes innate o...
1,14374db205f6934d9cba148624462000bc6ec7be,antibody treatment against angiopoietin-like 4...,"importance despite extensive global efforts, ..."
2,af678e8cd31d74cdb2d690addc19d59dca331f2b,quantifying the seasonal drivers of transmissi...,"growing human population, urbanization and gl..."
3,42b049c2b5b32c094dc8b10f967e43ac2169b890,evaluation of the influenza-like illness surve...,the first evaluation of the tunisian influenz...
4,1664a9df618ca74e099245a2bd65f3172aeac284,nan,"worldwide, lung cancer remains the most frequ..."


In [91]:
%%capture
!pip install transformers

In [90]:
%%capture
!pip install sentence-transformers

## Step 2.a - First sentence embedding approach

In [88]:
import torch

from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

## Step 2.b - Second sentence embedding approach

## Step 3 - Comparison of two models